## Методом к-ближайших соседей
### Лабораторная работа № 1
#### Задание 1
Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).

#### Задание 2
Провести классификацию найденного датасета, методом к- ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

file_path = "Electric_Vehicle_Population_Data.csv"
df = pd.read_csv(file_path)
df.dropna(inplace=True)

In [2]:
df

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,2C4RC1N71H,Kitsap,Bremerton,WA,98311.0,2017,CHRYSLER,PACIFICA,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,33,0,23.0,349437882,POINT (-122.6466274 47.6341188),PUGET SOUND ENERGY INC,5.303509e+10
1,2C4RC1N7XL,Stevens,Colville,WA,99114.0,2020,CHRYSLER,PACIFICA,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,32,0,7.0,154690532,POINT (-117.90431 48.547075),AVISTA CORP,5.306595e+10
2,KNDC3DLCXN,Yakima,Yakima,WA,98908.0,2022,KIA,EV6,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,14.0,219969144,POINT (-120.6027202 46.5965625),PACIFICORP,5.307700e+10
3,5YJ3E1EA0J,Kitsap,Bainbridge Island,WA,98110.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,23.0,476786887,POINT (-122.5235781 47.6293323),PUGET SOUND ENERGY INC,5.303509e+10
4,1N4AZ1CP7J,Thurston,Tumwater,WA,98501.0,2018,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,151,0,35.0,201185253,POINT (-122.89692 47.043535),PUGET SOUND ENERGY INC,5.306701e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159462,KM8JBDA2XP,Skamania,Underwood,WA,98651.0,2023,HYUNDAI,TUCSON,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,33,0,14.0,235949514,POINT (-121.5312858 45.7348285),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF S...,5.305995e+10
159463,1G1FZ6S02M,Skagit,Bow,WA,98232.0,2021,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,40.0,148544168,POINT (-122.440636 48.5613885),PUGET SOUND ENERGY INC,5.305795e+10
159464,YV4H60CX2P,King,Sammamish,WA,98029.0,2023,VOLVO,XC90,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,32,0,5.0,240200754,POINT (-121.9993659 47.5484866),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
159465,5YJ3E1EA7K,Whatcom,Bellingham,WA,98225.0,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220,0,40.0,156680590,POINT (-122.486115 48.761615),PUGET SOUND ENERGY INC||PUD NO 1 OF WHATCOM CO...,5.307300e+10


In [11]:
# Преобразование категориальной переменной в числовой формат
label_encoder = LabelEncoder()
df['Electric Vehicle Type'] = label_encoder.fit_transform(df['Electric Vehicle Type'])
df = df.sample(n=10000)

# Разделение на признаки (X) и целевую переменную (y)
X = df[['Postal Code', 'Model Year']]
y = df['Electric Vehicle Type']

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков (нормализация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
param_grid = {'n_neighbors': list(range(2, 11)), 'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}

# Создание модели k-ближайших соседей
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid, refit=True, verbose=0, cv=5)
# Обучение модели
grid.fit(X_train_scaled, y_train)

best_params = grid.best_params_
best_knn = grid.best_estimator_
# Предсказания на тестовой выборке с использованием лучшей модели
predictions = best_knn.predict(X_test_scaled)
# Оценка точности лучшей модели
accuracy = accuracy_score(y_test, predictions)
print(f"\nЛучшие параметры: {best_params}")
print(f"Точность: {accuracy}")

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))


Лучшие параметры: {'metric': 'euclidean', 'n_neighbors': 8}
Точность: 0.786

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.81      0.95      0.87      1573
           1       0.50      0.20      0.28       427

    accuracy                           0.79      2000
   macro avg       0.66      0.57      0.58      2000
weighted avg       0.75      0.79      0.75      2000

